In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets

In [2]:
data = pd.read_csv('../data/revised_weather.csv')

,dt,temp,feels_like,pressure,humidity,wind_speed,wind_deg,rain_1h,snow_1h,clouds_all,weather_main,weather_description,month,year,day,hour
0,1978-12-31 20:00:00,51.49,50.86,1015,96,12.91,176,NaN,NaN,100,Clouds,overcast clouds,12,1978,31,20
1,1978-12-31 21:00:00,51.55,50.92,1014,96,14.00,178,NaN,NaN,100,Clouds,overcast clouds,12,1978,31,21
2,1978-12-31 22:00:00,51.67,51.06,1014,96,14.81,185,NaN,NaN,100,Clouds,overcast clouds,12,1978,31,22
3,1978-12-31 23:00:00,54.54,53.96,1013,91,16.02,186,NaN,NaN,100,Clouds,overcast clouds,12,1978,31,23
4,1979-01-01 00:00:00,54.57,53.96,1013,90,16.13,187,NaN,NaN,100,Clouds,overcast clouds,1,1979,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373459,2021-08-08 15:00:00,85.62,91.08,1017,62,6.53,213,NaN,NaN,0,Clear,sky is clear,8,2021,8,15
373460,2021-08-08 16:00:00,86.13,95.38,1016,70,5.26,229,NaN,NaN,62,Clouds,broken clouds,8,2021,8,16
373461,2021-08-08 17:00:00,86.07,93.15,1016,65,5.26,229,NaN,NaN,62,Clouds,broken clouds,8,2021,8,17
373462,2021-08-08 18:00:00,85.78,92.91,1016,66,5.26,229,NaN,NaN,62,Clouds,broken clouds,8,2021,8,18
